In [1]:


%load_ext autoreload
%autoreload 2


In [2]:
import requests
import time

In [3]:
import logging
import contextlib
from http.client import HTTPConnection

def debug_requests_on():
    '''Switches on logging of the requests module.'''
    HTTPConnection.debuglevel = 1

    logging.basicConfig(filename = 'requestslog.log', format = '%(asctime)s %(message)s')
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

def debug_requests_off():
    '''Switches off logging of the requests module, might be some side-effects'''
    HTTPConnection.debuglevel = 0

    root_logger = logging.getLogger()
    root_logger.setLevel(logging.WARNING)
    root_logger.handlers = []
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.WARNING)
    requests_log.propagate = False

In [4]:
debug_requests_on()

In [4]:
## try current reqeust method
jubilee_ip = '192.168.1.2'

gcode_home = 'G28'


In [ ]:

reponse = requests.post(
                f"http://{jubilee_ip}/machine/code", data=f"{gcode_home}", timeout=5
            )

In [9]:
session = requests.session()
session.keep_alive = False

In [10]:
r = session.get(f"http://{jubilee_ip}/rr_model?key=seqs")

NameError: name 'jubilee_ip' is not defined

In [9]:
r.status_code

200

In [15]:
reply_response = session.get(
                    f"http://{jubilee_ip}/rr_model?key=seqs"
                )
logging.debug(f'MODEL response, status: {reply_response.status_code}, headers:{reply_response.headers}, content:{reply_response.content}')

reply_count = reply_response.json()["result"]["reply"]

send: b'GET /rr_model?key=seqs HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: application/json
header: Content-Length: 243
header: Connection: close


In [62]:
reply_count

16

In [16]:
buffer_response = session.get(
                    f"http://{jubilee_ip}/rr_gcode?gcode={gcode_home}", timeout=30
                )
logging.debug(f'GCODE response, status: {buffer_response.status_code}, headers:{buffer_response.headers}, content:{buffer_response.content}')

send: b'GET /rr_gcode?gcode=G28 HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: application/json
header: Content-Length: 12
header: Connection: close


In [7]:
reply_r

NameError: name 'buffer_response' is not defined

In [12]:
def delay_time(n):
    """
    Calculate delay time for next request. dumb hard code for now, could be fancy exponential backoff
    """
    if n == 0:
        return 0
    if n < 10:
        return 0.1
    if n < 20:
        return 0.2
    if n < 30:
        return 0.3
    else:
        return 1
    

In [18]:
tic = time.time()
response_wait = 60

try_count = 0
while True:
    try:
        response = session.get(
            f"http://{jubilee_ip}/rr_model?key=seqs"
        )
            logging.debug(f'MODEL response, status: {response.status_code}, headers:{response.headers}, content:{response.content}')
        print('Response: ', response)
        new_reply_count = response.json()["result"]["reply"]
        if new_reply_count != reply_count:
            response = session.get(
                f"http://{jubilee_ip}/rr_reply"
            )
            logging.debug(f'REPLY response, status: {response.status_code}, headers:{response.headers}, content:{response.content}')
            response = response.text
            break
        elif time.time() - tic > response_wait:
            response = None
            break
        print(f'Try count: {try_count}, delay time: {delay_time(try_count)} s')
        time.sleep(delay_time(try_count))
        try_count += 1
    except Exception as e:
        print(f"Connection error {e}, sleeping 1 second")
        logging.debug(f'Error in gcode reply wait loop: {e}')
        time.sleep(2)
        continue

send: b'GET /rr_model?key=seqs HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: application/json
header: Content-Length: 243
header: Connection: close
Response:  <Response [200]>
send: b'GET /rr_reply HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: text/plain
header: Content-Length: 0
header: Connection: close


In [46]:
response.headers

{'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Content-Type': 'text/plain', 'Content-Length': '0', 'Connection': 'close'}

In [44]:
response = session.get(
                                f"http://{jubilee_ip}/rr_reply"
                            )

send: b'GET /rr_reply HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: text/plain
header: Content-Length: 0
header: Connection: close


In [26]:
response.text

''

In [15]:
from science_jubilee import Machine

In [16]:
jub = Machine.Machine(address = '192.168.1.2')

send: b'POST /machine/code HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 25\r\n\r\n'
send: b'M409 K"move.axes[].homed"'
reply: 'HTTP/1.1 500 only rr_upload is supported for POST requests\r\n'
header: Connection: close
send: b'GET /rr_model?key=seqs HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: close\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cache-Control: no-cache, no-store, must-revalidate
header: Pragma: no-cache
header: Expires: 0
header: Content-Type: application/json
header: Content-Length: 243
header: Connection: close
send: b'GET /rr_gcode?gcode=M409%20K%22move.axes%5B%5D.homed%22 HTTP/1.1\r\nHost: 192.168.1.2\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: close\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Cac

In [ ]:
jub.home_all()

In [17]:
jub.session.headers

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'close'}